In [3]:
# 必要なライブラリをインストール（Google Colabでは初回実行時に必要）
# !pip install transformers torch Pillow requests

# ライブラリをインポート
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import torch

# 1. CLIPモデルとプロセッサの読み込み
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 2. 画像をURLから読み込む
image_url ="https://kyoto-edu.sakura.ne.jp/althusser/img/fox.jpg"  # ここに有効な画像URLを指定
try:
    response = requests.get(image_url)
    response.raise_for_status()  # URLが無効な場合にエラーを発生
    image = Image.open(BytesIO(response.content))
except requests.exceptions.RequestException as e:
    print(f"Error fetching image from URL: {e}")
    exit()
except Exception as e:
    print(f"Error processing image: {e}")
    exit()

# 3. キャプション候補を定義
candidate_captions = [
    "A shark stuffed animal",
    "A Fox stuffed animal",
    "A Triceratops stuffed animal",
    "A person reading a book",
    "A cup of coffee on a table"
]

# 4. 画像とキャプションを処理
inputs = processor(text=candidate_captions, images=image, return_tensors="pt", padding=True)

# 5. モデルで画像とキャプションのマッチングスコアを計算
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

# 6. 結果を表示
print("キャプションと確率:")
for caption, prob in zip(candidate_captions, probs[0]):
    print(f"{caption}: {prob:.4f}")

# 7. 最も確率の高いキャプションを選択
best_caption_idx = probs.argmax().item()
best_caption = candidate_captions[best_caption_idx]
print(f"\n最も適切なキャプション: {best_caption} (確率: {probs[0][best_caption_idx]:.4f})")

# 8. 画像を表示（確認用、Google Colabでは表示可能）
image.show()

キャプションと確率:
A shark stuffed animal: 0.1887
A Fox stuffed animal: 0.5170
A Triceratops stuffed animal: 0.2943
A person reading a book: 0.0000
A cup of coffee on a table: 0.0000

最も適切なキャプション: A Fox stuffed animal (確率: 0.5170)
